In [1]:
import numpy as np
import os
import xml.etree.ElementTree as ET
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.model_selection import train_test_split

In [2]:
projectDir = r'C:\Users\Viktor\Skola\embedded\embedded_project'
projectDir2 = r'C:\Users\WillyB\Documents\GitHub\embedded_project'

homeDir = os.path.join(projectDir2,'keras_Realtime_Multi-Person_Pose_Estimation-master')
xmlDir = os.path.join(projectDir2, 'Stanford40_XMLAnnotations\XMLAnnotations')

#homeDir = r'C:\Users\Viktor\Skola\embedded\embedded_project\keras_Realtime_Multi-Person_Pose_Estimation-master'
#xmlDir = r'C:\Users\Viktor\Skola\embedded\embedded_project\Stanford40_XMLAnnotations\XMLAnnotations'

In [3]:
x = np.load(os.path.join(homeDir,'partCoords.npy'))
print(x.shape)

(9532, 18, 2)


In [4]:
#np.set_printoptions(suppress=True)

#x10 = x[:10]

#print(x10)

#x10_norm = np.zeros(x10.shape)

for i in range(len(x)):
    #print(x10[i])
    xmax, ymax = np.nanmax(x[i], axis=0)
    xmin, ymin = np.nanmin(x[i], axis=0)
    
    #if np.isnan(xmax):
    #    print(x[i])
    #print(i)
    #print(xmin)
    #print(xmax)
    #print(ymin)
    #print(ymax)
    
    x[i][:,0] -= xmin
    x[i][:,0] /= (xmax - xmin)
    
    x[i][:,1] -= ymin
    x[i][:,1] /= (ymax - ymin)
    
    #print(x[i])
#print(x10_norm)
#print(x[1297])

C:\Users\WillyB\anaconda3\envs\gputest\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: All-NaN slice encountered
  # This is added back by InteractiveShellApp.init_path()
C:\Users\WillyB\anaconda3\envs\gputest\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: All-NaN slice encountered
  if sys.path[0] == '':


In [5]:
x[np.isnan(x)] = 0

In [6]:
print(x)

[[[0.54545455 0.04210526]
  [0.4171123  0.29824561]
  [0.07486631 0.34035088]
  ...
  [0.55080214 0.        ]
  [0.25668449 0.08070175]
  [0.         0.        ]]

 [[0.46       0.05291005]
  [0.445      0.47619048]
  [0.1        0.49206349]
  ...
  [0.525      0.00529101]
  [0.265      0.1005291 ]
  [0.6        0.11640212]]

 [[0.39344262 0.11111111]
  [0.78688525 0.51111111]
  [1.         0.43333333]
  ...
  [0.45901639 0.01111111]
  [0.         0.        ]
  [0.74590164 0.        ]]

 ...

 [[0.78723404 0.29166667]
  [0.56382979 0.48611111]
  [0.24468085 0.40277778]
  ...
  [0.84574468 0.22916667]
  [0.57978723 0.        ]
  [0.         0.        ]]

 [[0.53784861 0.18315018]
  [0.60756972 0.27838828]
  [0.35258964 0.23076923]
  ...
  [0.62749004 0.10989011]
  [0.46414343 0.        ]
  [0.80278884 0.        ]]

 [[0.24418605 0.2755102 ]
  [0.70930233 0.34693878]
  [0.38372093 0.32653061]
  ...
  [0.34883721 0.15306122]
  [0.         0.        ]
  [0.80232558 0.        ]]]


In [7]:
print(x[0].shape)

(18, 2)


In [8]:
os.chdir(homeDir)

jpgOrder = open('JPG_Order.txt', 'r')
lines = jpgOrder.readlines()

y_string = []

os.chdir(xmlDir)

for i in range(len(lines)):
    imgFileName = lines[i].strip()
    fileName = imgFileName.split(".")[0]
    xmlFileName = fileName + ".xml"
    
    root = ET.parse(xmlFileName).getroot()
    action = root.find('object/action').text
    
    y_string.append(action)
    
    print(str(i) + '/' + str(len(lines)),end='\r')

#print(y_string)

In [9]:
(allUniqueClasses,y_int) = np.unique(y_string, return_inverse=True)

print(y_int)

y_cat = tf.keras.utils.to_categorical(y_int, len(allUniqueClasses))

print(y_cat)

[ 0  0  0 ... 39 39 39]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y_cat, test_size=0.2, random_state=42)

In [11]:
def defineModel(nrOfClasses):
    model = Sequential()
    model.add(keras.Input(shape=(18,2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    
    model.add(Dense(nrOfClasses, activation='softmax'))
    
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [12]:
model = defineModel(40)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 36)                0         
_________________________________________________________________
dense (Dense)                (None, 256)               9472      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 40)                10280     
Total params: 85,544
Trainable params: 85,544
Non-trainable params: 0
_________________________________________________________________


In [13]:
#x = x[np.newaxis,...]

In [14]:
#y_cat = y_cat[np.newaxis,...]

In [15]:
model.fit(x_train,y_train,epochs=20,verbose=1)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
239/239 [==============================] - 0s 2ms/step - loss: 3.2010 - accuracy: 0.1464
Epoch 2/20
239/239 [==============================] - 0s 2ms/step - loss: 2.8063 - accuracy: 0.2332
Epoch 3/20
239/239 [==============================] - 0s 2ms/step - loss: 2.6366 - accuracy: 0.2675
Epoch 4/20
239/239 [==============================] - 0s 2ms/step - loss: 2.51

In [16]:
model.evaluate(x_test,y_test)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
60/60 [==============================] - 0s 1ms/step - loss: 2.5430 - accuracy: 0.3340


[2.5430428981781006, 0.33403250575065613]

In [17]:
import itertools

In [18]:
#General function that creates a Neural network model based on the input.
def createModel(denseHiddenLayer, outputLayer, inShape, nrOfclasses):
    
    model = Sequential()
    #input layer
    model.add(keras.Input(shape=inShape))
    model.add(Flatten())
    
    #hidden layers
    print(len(denseHiddenLayer))
    for layer in denseHiddenLayer:
        model.add(Dense(layer[0], activation=layer[1], kernel_initializer='he_uniform'))

    #outputlayer
    model.add(Dense(nrOfclasses, activation = outputLayer))
    
    #model.summary()
    return model

In [19]:

#User decide the input layer settings: use intergets only and not float 
#The numbers of neurons to test in the input layer

#user selects different hidden layers to try
nrHiddenLayers = [3,4]
#user selects the different neurons to try
nrHiddenLayersNeurons = [128,256,512]
#user selects the different activatons to try
hiddenLayersActivations = ['relu']
#combine every combination for hidden layers
nrHiddenLayersCombination = list(itertools.product(*[nrHiddenLayersNeurons, hiddenLayersActivations]))

#User select the output layers activation to try
outputLayerActivations = ['softmax']

#user selects SGD's learning rate (interget and float are valid)
learningRate = [0.001, 0.0001]

In [20]:
#This is an important function that creates every combinations for the NN model based on the selected data from the user.
def allModelData(nrhidLayers, nrhidLayersComb, outLayerAct, lRate ):
    
    hiddenLayerCombinations = []
    for i in nrhidLayers:
        temp = []
        for x in range(i):
            temp.append(nrhidLayersComb)
        hiddenLayerCombinations.append(temp)
    
    allHiddenLayerCombination = []
    for x in hiddenLayerCombinations:
        combb = list(itertools.product(*x))
        HiddenLayerCombination = list(itertools.product(*[combb, outLayerAct, lRate]))
        allHiddenLayerCombination.append(HiddenLayerCombination)
        
    print(len(allHiddenLayerCombination[0]))
    print(len(allHiddenLayerCombination[1]))

    
    all_model_combinations = list(itertools.chain.from_iterable(allHiddenLayerCombination))
    print(len(all_model_combinations))
    return all_model_combinations


all_model_combination = allModelData(nrHiddenLayers, nrHiddenLayersCombination, outputLayerActivations, learningRate )
#all_model_combination[:10]
print(len(all_model_combination))


54
162
216
216


In [21]:
print(all_model_combination[0][0])
print(all_model_combination[0][1])
print(all_model_combination[0][2])


((128, 'relu'), (128, 'relu'), (128, 'relu'))
softmax
0.001


In [22]:
#list of the score and the model settings will be stored here 
results =[]
#Iterate through every combinations and appends the score and model settings in the list results.
count = 1
for attributes in all_model_combination:
    print(count,"of ",len(all_model_combination))
    count += 1
    denseHiddenLayer_data = attributes[0]
    outputLayer_data = attributes[1]
    learningRate_data = attributes[2]
    print(denseHiddenLayer_data,outputLayer_data,learningRate_data)
    nn_model = createModel(denseHiddenLayer_data, outputLayer_data, (18,2), 40)
    opt = keras.optimizers.Adam(learning_rate=learningRate_data)
    #optimizer =keras.optimizers.SGD(learning_rate = learningRate_data)
    nn_model.compile(loss= keras.losses.categorical_crossentropy,
              optimizer =opt,
              metrics =['accuracy'] )
    nn_model.fit(x_train,y_train,
          batch_size = 32,
          epochs = 20, verbose=0)
    score = nn_model.evaluate(x_train,y_train, verbose = 0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    results.append([(denseHiddenLayer_data, outputLayer_data, learningRate_data), score[1]])

1 of  216
((128, 'relu'), (128, 'relu'), (128, 'relu')) softmax 0.001
3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_con

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Test loss: 1.149465799331665
Test accuracy: 0.6533770561218262
6 of

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Test loss: 1.1739023923873901
Test accuracy: 0.6478688716888428
10 of  216
((128, 'relu'), (256, 'relu'), (256, 'relu')) softmax 0.0001
3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Test loss: 1.20072340965271
Test accuracy: 0.641967236995697
14 of  216
((128, 'relu'), (512, 'relu'), (128, 'relu')) softmax 0.0001
3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, d

Test loss: 0.6521548628807068
Test accuracy: 0.8023606538772583
18 of  216
((128, 'relu'), (512, 'relu'), (512, 'relu')) softmax 0.0001
3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Test loss: 1.904791235923767
Test accuracy: 0.4520655870437622
197 of  216
((512, 'relu'), (256, 'relu'), (512, 'relu'), (512, 'relu')) softmax 0.001
4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silenc

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Test loss: 2.073314905166626
Test accuracy: 0.4068196713924408
201 of  216
((512, 'relu'), (512, 'relu'), (128, 'relu'), (256, 'relu')) softmax 0.001
4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silenc

Test loss: 1.8825711011886597
Test accuracy: 0.45718032121658325
205 of  216
((512, 'relu'), (512, 'relu'), (256, 'relu'), (128, 'relu')) softmax 0.001
4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To sile

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Test loss: 0.5181297063827515
Test accuracy: 0.8372458815574646
210

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Test loss: 0.49827203154563904
Test accuracy: 0.8448524475097656
214 of  216
((512, 'relu'), (512, 'relu'), (512, 'relu'), (256, 'relu')) softmax 0.0001
4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To sil

In [23]:
#Show top 10 highest score (Accuracy) and the setting used.
def takeSecond(elem):
    return elem[1]
top10 = sorted(results, key= takeSecond, reverse = True)[:10]

In [25]:
top10

[[(((512, 'relu'), (512, 'relu'), (512, 'relu'), (512, 'relu')),
   'softmax',
   0.001),
  0.8815737962722778],
 [(((256, 'relu'), (512, 'relu'), (512, 'relu'), (512, 'relu')),
   'softmax',
   0.001),
  0.8771147727966309],
 [(((256, 'relu'), (256, 'relu'), (512, 'relu'), (512, 'relu')),
   'softmax',
   0.001),
  0.8737049102783203],
 [(((128, 'relu'), (512, 'relu'), (512, 'relu'), (512, 'relu')),
   'softmax',
   0.001),
  0.8710819482803345],
 [(((512, 'relu'), (256, 'relu'), (512, 'relu'), (512, 'relu')),
   'softmax',
   0.001),
  0.8641311526298523],
 [(((128, 'relu'), (256, 'relu'), (512, 'relu'), (512, 'relu')),
   'softmax',
   0.001),
  0.8625573515892029],
 [(((256, 'relu'), (512, 'relu'), (512, 'relu'), (256, 'relu')),
   'softmax',
   0.001),
  0.8574426174163818],
 [(((128, 'relu'), (512, 'relu'), (256, 'relu'), (512, 'relu')),
   'softmax',
   0.001),
  0.8553442358970642],
 [(((128, 'relu'), (512, 'relu'), (512, 'relu'), (256, 'relu')),
   'softmax',
   0.001),
  0.85

In [26]:
sortedAccuracy = sorted(results, key= takeSecond, reverse = True)

In [30]:
np.save(homeDir+"/sortedAccuracy.npy", np.array(sortedAccuracy))